#### Now that we have our designs of experiment, we can begin to create our metamodels. In our case, we will create 3 metamodels, each created from a different (in size and elements) design of experiment. We will then assess the precision of our metamodel in regard to the functions ouput, and further we will look at the sensitivities

In [1]:
import pandas as pd
import numpy as np
import openturns as ot
import pythontools as pt
from collections import Sequence, Iterable
import KarhunenLoeveFieldSensitivity as klfs

/home/kramer84/anaconda3/envs/stochastic_field_env/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  """


In [2]:
# Lets' first load our 3 designs of experiment for the kriging models. 
# input designs : 
doe50_in = pd.read_csv('DOE_RESP2/kg_doe50_130875.csv', sep=';')
doe100_in = pd.read_csv('DOE_RESP2/kg_doe100_409484.csv', sep=';')
doe200_in = pd.read_csv('DOE_RESP2/kg_doe200_224409.csv', sep=';')
# model outputs :
doe50_out_MD = pd.read_csv('DOE_RESP2/kg_doe50_130875_resp_MD.csv', sep=';', header=None)
doe50_VM = pd.read_csv('DOE_RESP2/kg_doe50_130875_resp_VM.csv', sep=';', header=None)
doe100_out_MD = pd.read_csv('DOE_RESP2/kg_doe100_409484_resp_MD.csv', header=None)
doe100_VM = pd.read_csv('DOE_RESP2/kg_doe100_409484_resp_VM.csv', sep=';', header=None)
doe200_out_MD = pd.read_csv('DOE_RESP2/kg_doe200_224409_resp_MD.csv', sep=';', header=None)
doe200_VM = pd.read_csv('DOE_RESP2/kg_doe200_224409_resp_VM.csv', sep=';', header=None)
# validation models : 
doe_vali_200_in = pd.read_csv('DOE_RESP2/val_doe200_502849.csv', sep=';')
doe_vali_200_out_MD = pd.read_csv('DOE_RESP2/val_doe200_502849_resp_MD.csv', sep=';',header=None)
doe_vali_200_out_VM = pd.read_csv('DOE_RESP2/val_doe200_502849_resp_VM.csv', sep=';',header=None)

FileNotFoundError: [Errno 2] No such file or directory: 'DOE_RESP2/kg_doe50_130875.csv'

In [ ]:
def all_same(items=None):
    #Checks if all items of a list are the same
    return all(x == items[0] for x in items)

def isValidSobolIndicesExperiment(sample_like, size, second_order = False):
    try :
        sample = np.asarray(sample_like)
    except :
        print('Could not convert sample to numpy array')
        raise TypeError
    N = sample.shape[0]
    print('N is', N)
    dim = sample.shape[1]
    Y_A = sample[:size]
    Y_B = sample[size:2*size]
    N_indice = int(N/size - 2)
    assert N%size==0,"wrong sample size"
    print('Simplified view of the sobol indices experiments structure')
    print('There are {} dimensions and {} sobol indices to calculate'.format(dim, N_indice))
    assert np.where(Y_A==Y_B,True,False).any() == False #Here we check that there is no similarity at all  between A and B 
    tot_lines = list()
    try : 
        for i in range(N_indice+2):
            Y_E = sample[i*size:(i+1)*size]
            tot_cols = [True]*dim
            col_where_A = list(set(np.argwhere(Y_A==Y_E)[:,1]))
            line_where_A = list(set(np.argwhere(Y_A==Y_E)[:,0]))
            if len(line_where_A)==size : OK = True
            for co in col_where_A :
                tot_cols[co] = False
            if OK : 
                if len(col_where_A)==dim and all(tot_cols):
                    sl = ['A_'+str(j) for j in range(dim)]
                elif len(col_where_A)==0 and all_same(tot_cols):
                    sl = ['B_'+str(j) for j in range(dim)]
                else :
                    sl = ['B_'+str(j) for j in range(dim)]
                    for k in range(len(col_where_A)):
                        sl[col_where_A[k]] = 'A_'+str(col_where_A[k])
                l = "  ,    ".join(sl)
                l = '    '+l
            if not OK :
                print('Error')
                return False
            tot_lines.append(l)
        repres = ' \n\n'.join(tot_lines)
        repres = '\n'+repres
        print(repres) 
        return True
    except :
        return False

def ereaseNanFromSample(sample_in, sample_out, N , secondOrder = False):
    sampOut = np.array(sample_out,copy=True, subok=False)
    if secondOrder == False : 
        n_vars = int(sampOut.shape[0]/N) - 2 
        print('n vars is', n_vars)
        N_max = int(N*(n_vars + 2))
        N_var = N_max/N
    else : 
        n_vars = int(sampOut.shape[0]/(N*2)) - 1
        print('n vars is', n_vars)
        N_max = int(N*(2*n_vars + 2))
        N_var = N_max/N
    
    print('N_max is', N_max)
    argNan = np.argwhere(np.isnan(sampOut))[:,0].tolist()
    print('args where nan : ',argNan)
    toErease = set()
    for arg in argNan:
        whereToErease = list(range(arg%N, N_max, N))
        [toErease.add(elem) for elem in whereToErease]
    whereToErease = list(toErease)
    print('Where we are erasing:', whereToErease)
    N -= int(len(whereToErease)/N_var)
    for idx in sorted(whereToErease)[::-1]:
        sample_in.erase(idx)
        sample_out.erase(idx) 
    print('N is now: ',N)
    return N

In [ ]:
# KRIGING SAMPLE
    # INPUTS
sample_doe50_in = ot.Sample(doe50_in.values)
sample_doe50_in.setDescription(doe50_in.columns)
sample_doe100_in = ot.Sample(doe100_in.values)
sample_doe100_in.setDescription(doe100_in.columns)
sample_doe200_in = ot.Sample(doe200_in.values)
sample_doe200_in.setDescription(doe200_in.columns)
    # OUTPUTS
sample_doe50_out_MD = ot.Sample(doe50_out_MD.values)
sample_doe50_out_MD.setDescription(ot.Description.BuildDefault(len(doe50_out_MD.columns), 'MD_'))
sample_doe50_VM = ot.Sample(doe50_VM.values)
sample_doe50_VM.setDescription(ot.Description.BuildDefault(len(doe50_VM.columns), 'VM_'))
sample_doe100_out_MD = ot.Sample(doe100_out_MD.values)
sample_doe100_out_MD.setDescription(ot.Description.BuildDefault(len(doe100_out_MD.columns), 'MD_'))
sample_doe100_VM = ot.Sample(doe100_VM.values)
sample_doe100_VM.setDescription(ot.Description.BuildDefault(len(doe100_VM.columns), 'VM_'))
sample_doe200_out_MD = ot.Sample(doe200_out_MD.values)
sample_doe200_out_MD.setDescription(ot.Description.BuildDefault(len(doe200_out_MD.columns), 'MD_'))
sample_doe200_VM = ot.Sample(doe200_VM.values)
sample_doe200_VM.setDescription(ot.Description.BuildDefault(len(doe200_VM.columns), 'VM_'))
# VALIDATION SAMPLE
validation_sample_doe200_in = ot.Sample(doe_vali_200_in.values)
validation_sample_doe200_in.setDescription(doe_vali_200_in.columns)
validation_sample_doe200_out_MD = ot.Sample(doe_vali_200_out_MD.values)
validation_sample_doe200_out_MD.setDescription(ot.Description.BuildDefault(len(doe_vali_200_out_MD.columns), 'MD_'))
validation_sample_doe200_out_VM = ot.Sample(doe_vali_200_out_VM.values)
validation_sample_doe200_out_VM.setDescription(ot.Description.BuildDefault(len(doe_vali_200_out_VM.columns), 'VM_'))
sample_doe50_in.getDescription()

## First, let's define a metamodeling class

In [ ]:
class metamodeling_kriging : 
    def __init__(self, inSample, outSample, **kwargs):
        self.input_sample = inSample
        self.output_sample = outSample
        self.__default_kriging__ = None
        self.__kriging_theta__ = None
        self.__kriging_results__ = None
        self.__kriging_metamodel__ = None
        self.__size_multistart__ = kwargs['size_multistart'] if 'size_multistart' in kwargs else 5
        self.__lb__ = kwargs['lower_bound'] if 'lower_bound' in kwargs else None
        self.__ub__ = kwargs['upper_bound'] if 'upper_bound' in kwargs else None
        self.__optim_type__ = kwargs['optim_type'] if 'optim_type' in kwargs else 'best_start'
        self.validation_results = __validation_results__() 
        
    def _build_default(self):
        self.__default_kriging__ = pt.build_default_kriging_algo(
                                    input_sample  = self.input_sample,
                                    output_sample = self.output_sample, 
                                    basis         = None,
                                    covariance_model = None,
                                    noise         = None)
        
    def _estimate_theta(self):
        self.__kriging_theta__ = pt.estimate_kriging_theta(
                            algo_kriging = self.__default_kriging__,
                            lower_bound = self.__lb__,
                            upper_bound = self.__ub__,
                            size        = self.__size_multistart__,
                            optim_type  = self.__optim_type__)
    
    def _get_results_metamodel(self):
        if isinstance(self.__kriging_theta__,(Sequence,Iterable, list)):
            self.__kriging_results__ = [_kt.getResult() for _kt in self.__kriging_theta__]
            self.__kriging_metamodel__ = [_km.getMetaModel() for _km in self.__kriging_results__]
        else :
            self.__kriging_results__ = self.__kriging_theta__.getResult()
            self.__kriging_metamodel__ = self.__kriging_results__.getMetaModel()
                
    def run(self):
        self._build_default()
        self._estimate_theta()
        self._get_results_metamodel()
        print('Done !')
        
    def getKrigingResult(self):
        return self.__kriging_results__
        
    def getKrigingMetaModel(self):
        return self.__kriging_metamodel__
    
    def _check_clean_nans(self, sampleIn, sampleOut):
        whereNan = list(set(np.argwhere(np.isnan(sampleOut))[:,0]))
        print('NaN values found at index:',whereNan)
        [(sampleOut.erase(int(val)), sampleIn.erase(int(val))) for val in whereNan]
        
    def getMetaModelValidation(self, sample_in_validation, sample_out_validation):
        assert self.__kriging_metamodel__ is not None, "Please first run calculus"
        assert len(sample_in_validation) == len(sample_out_validation)
        self._check_clean_nans(sample_in_validation, sample_out_validation)
        self.validation_results.clear()
        if isinstance(self.__kriging_metamodel__,(Sequence,Iterable, list)):
            for i, model in enumerate(self.__kriging_metamodel__):
                validation = ot.MetaModelValidation(sample_in_validation, 
                                                    sample_out_validation[:,i],
                                                    self.__kriging_metamodel__[i])
                R2 = validation.computePredictivityFactor()
                residual = validation.getResidualSample()
                graph = validation.drawValidation()
                self.validation_results.addGraph(graph)
                self.validation_results.addR2(R2)
                self.validation_results.addResidual(residual)
        else : 
            validation = ot.MetaModelValidation(sample_in_validation, 
                                                sample_out_validation[:,0],
                                                self.__kriging_metamodel__)
            R2 = validation.computePredictivityFactor()
            residual = validation.getResidualSample()
            graph = validation.drawValidation()
            self.validation_results.addGraph(graph)
            self.validation_results.addR2(R2)
            self.validation_results.addResidual(residual)
                
class __validation_results__(object) :
    def __init__(self): 
        self.__R2__ = []
        self.__residuals__ = []
        self.__graphs__ = []

    def clear(self):
        self.__R2__.clear()
        self.__residuals__.clear()
        self.__graphs__.clear()
        
    def addGraph(self, graph):
        self.__graphs__.append(graph)

    def addR2(self, R2):
        self.__R2__.append(R2)

    def addResidual(self, residual):
        self.__residuals__.append(residual)

    def getGraphs(self):
        for graph in self.__graphs__ :
            ot.Show(graph)

    def getResiduals(self):
        theGraph = ot.Graph('Residuals','varying dimension','residual',True,'')
        theCurve = ot.Curve(list(range(len(self.__residuals__))),
                            self.__residuals__, 'residuals')
        theGraph.add(theCurve)
        ot.Show(theGraph)

    def getR2s(self):
        theGraph = ot.Graph('R2','varying dimension','residual',True,'')
        theCurve = ot.Curve(list(range(len(self.__R2__))),
                            self.__R2__, 'R2')
        theGraph.add(theCurve)
        ot.Show(theGraph)

#### First metamodel, DOE : LHS 50  ==> Max Deflection

In [ ]:
kriging_doe_50_MD = metamodeling_kriging(sample_doe50_in, sample_doe50_out_MD,optim_type='multi_start', size_multistart = 10)
kriging_doe_50_MD.run()

###### For validation, we are going to use one of the samples intended for the sobol indices calculus. by the way, we are only going to use the 2/7 th of the samples (as we have samples A, B, and then 5 combinations of those.)

In [ ]:
# validation_sample_doe2000_in | validation_sample_doe2000_out_MD | validation_sample_doe2000_out_VM
kriging_doe_50_MD.getMetaModelValidation(validation_sample_doe200_in, validation_sample_doe200_out_MD)


In [ ]:
kriging_doe_50_MD.validation_results.getGraphs()

###### As we see, the results here are not so horrible ...

#### Second metamodel, DOE : LHS 100  ==> Max Deflection

In [ ]:
kriging_doe_100_MD = metamodeling_kriging(sample_doe100_in, 
                                        sample_doe100_out_MD, 
                                        optim_type='best_start',
                                        size_multistart = 100,
                                        lower_bound = None,
                                        upper_bound = None)
kriging_doe_100_MD.run()

In [ ]:
kriging_doe_100_MD.getMetaModelValidation(validation_sample_doe200_in, validation_sample_doe200_out_MD)


In [ ]:

kriging_doe_100_MD.validation_results.getGraphs()

#### Third metamodel, DOE : LHS 200  ==> Max Deflection

In [ ]:
kriging_doe_200_MD = metamodeling_kriging(sample_doe200_in, 
                                        sample_doe200_out_MD, 
                                        optim_type='best_start',
                                        size_multistart = 100,

                                        lower_bound = None,
                                        upper_bound = None)
kriging_doe_200_MD.run()

In [ ]:
kriging_doe_200_MD.getMetaModelValidation(validation_sample_doe200_in, validation_sample_doe200_out_MD)


In [ ]:
kriging_doe_200_MD.validation_results.getGraphs()

## Comparison of the sensitivity analysis parts. Let's do the sensitivty analysis of the 3 metamodels.

We will first have to load the sample intended to the calculus of the sobol indices, in hope we will find no nan. 

In [ ]:
si_2000_in = pd.read_csv('DOE_RESP2/si_doe2000_720442.csv', sep=';')
si_2000_out_MD = pd.read_csv('DOE_RESP2/si_doe2000_720442_resp_MD.csv', sep=';',header=None)
si_2000_out_VM = pd.read_csv('DOE_RESP2/si_doe2000_720442_resp_VM.csv', sep=';',header=None)
#Samples
sample_si_2000_in = ot.Sample(si_2000_in.values)
sample_si_2000_in.setDescription(si_2000_in.columns)

sample_si_2000_out_MD = ot.Sample(si_2000_out_MD.values)
sample_si_2000_out_MD.setDescription(ot.Description.BuildDefault(len(si_2000_out_MD.columns), 'MD_'))

sample_si_2000_out_VM = ot.Sample(si_2000_out_VM.values)
sample_si_2000_out_VM.setDescription(ot.Description.BuildDefault(len(si_2000_out_VM.columns), 'VM_'))

si_1000_in = pd.read_csv('DOE_RESP2/si_doe1000_248214.csv', sep=';')
si_1000_out_MD = pd.read_csv('DOE_RESP2/si_doe1000_248214_resp_MD.csv', sep=';',header=None)
si_1000_out_VM = pd.read_csv('DOE_RESP2/si_doe1000_248214_resp_VM.csv', sep=';',header=None)
#Samples
sample_si_1000_in = ot.Sample(si_1000_in.values)
sample_si_1000_in.setDescription(si_1000_in.columns)

sample_si_1000_out_MD = ot.Sample(si_1000_out_MD.values)
sample_si_1000_out_MD.setDescription(ot.Description.BuildDefault(len(si_1000_out_MD.columns), 'MD_'))

sample_si_1000_out_VM = ot.Sample(si_1000_out_VM.values)
sample_si_1000_out_VM.setDescription(ot.Description.BuildDefault(len(si_1000_out_VM.columns), 'VM_'))
sample_si_2000_in.getSize()

In [ ]:
N_2000 = ereaseNanFromSample(sample_si_2000_in, sample_si_2000_out_MD, 2000, False)
sample_si_2000_in.getSize()


In [ ]:
sample_si_2000_in[7777,5]=256
isValidSobolIndicesExperiment(sample_si_2000_in,N_2000)


In [ ]:
kriging_doe_200_MD.getMetaModelValidation(sample_si_2000_in, sample_si_2000_out_MD)
kriging_doe_200_MD.validation_results.getGraphs()
#Astuce / plan d'experience en augmentant coeff de variation des variables

In [ ]:
N_1000 = ereaseNanFromSample(sample_si_1000_in, sample_si_1000_out_MD, 1000, False)

In [ ]:
kriging_doe_200_MD.getMetaModelValidation(sample_si_1000_in, sample_si_1000_out_MD)
kriging_doe_200_MD.validation_results.getGraphs()

In [ ]:
sensitivityAnalysis_REAL_MODEL_1000 = klfs.SobolKarhunenLoeveFieldSensitivityAlgorithm()
sensitivityAnalysis_REAL_MODEL_1000.setDesign(sample_si_1000_in, sample_si_1000_out_MD, N_1000)
sensitivityAnalysis_REAL_MODEL_1000.setEstimator(ot.SaltelliSensitivityAlgorithm())

In [ ]:
import spsa 


In [ ]:
analysis = spsa.SobolIndicesStochasticProcessAlgorithm(np.array(sample_si_1000_out_MD),N_1000)
analysis.getFirstOrderIndices()   ##### THE SOBOL INDICES CALCULUS IS RIGHT!!!! 

In [ ]:
sensitivityAnalysis_REAL_MODEL_1000.getFirstOrderIndices()[0]

In [ ]:
sensitivityAnalysis_REAL_MODEL_1000.__results__[0].getFirstOrderIndicesDistribution().getMarginal(0).getStandardDeviation()
res = sensitivityAnalysis_REAL_MODEL_1000.__results__[0]

In [ ]:
sensitivityAnalysis_REAL_MODEL_1000.getFirstOrderIndicesInterval()[0]

In [ ]:
sensitivityAnalysis_REAL_MODEL_1000.getTotalOrderIndices()[0]

In [ ]:
sensitivityAnalysis_REAL_MODEL_2000 = klfs.SobolKarhunenLoeveFieldSensitivityAlgorithm()
sensitivityAnalysis_REAL_MODEL_2000.setDesign(sample_si_2000_in, sample_si_2000_out_MD, N_2000)
sensitivityAnalysis_REAL_MODEL_2000.setEstimator(ot.SaltelliSensitivityAlgorithm())
sensitivityAnalysis_REAL_MODEL_2000.getFirstOrderIndices()[0]

In [ ]:
si_8000_in = pd.read_csv('DOE_RESP2/si_doe8000_439038.csv', sep=';')
si_8000_out_MD = pd.read_csv('DOE_RESP2/si_doe8000_439038_resp_MD.csv', sep=';',header=None)
si_8000_out_VM = pd.read_csv('DOE_RESP2/si_doe8000_439038_resp_VM.csv', sep=';',header=None)
#Samples
sample_si_8000_in = ot.Sample(si_8000_in.values)
sample_si_8000_in.setDescription(si_8000_in.columns)

sample_si_8000_out_MD = ot.Sample(si_8000_out_MD.values)
sample_si_8000_out_MD.setDescription(ot.Description.BuildDefault(len(si_8000_out_MD.columns), 'MD_'))

sample_si_8000_out_VM = ot.Sample(si_8000_out_VM.values)
sample_si_8000_out_VM.setDescription(ot.Description.BuildDefault(len(si_8000_out_VM.columns), 'VM_'))

In [ ]:
whereNan = np.argwhere(np.isnan(sample_si_8000_out_MD))[:,0].tolist()
N = 8000
N_MAX = 8000*7
toDelete = set()
print('Where nan:', whereNan)
for idx in whereNan:
    toErease = list(range(idx%8000,N_MAX,N))
    [toDelete.add(elem) for elem in toErease]
print('toErease is', toDelete)
toDelete = list(toDelete)
N-=int(len(toDelete)/7)
for idx in sorted(toDelete)[::-1] :
    sample_si_8000_in.erase(idx)
    sample_si_8000_out_MD.erase(idx)
print('N=',N)

In [ ]:
sensitivityAnalysis_REAL_MODEL2 = klfs.SobolKarhunenLoeveFieldSensitivityAlgorithm()
sensitivityAnalysis_REAL_MODEL2.setDesign(sample_si_8000_in, sample_si_8000_out_MD, N)
sensitivityAnalysis_REAL_MODEL2.setEstimator(ot.SaltelliSensitivityAlgorithm())

In [ ]:
sensitivityAnalysis_REAL_MODEL2.getFirstOrderIndices()

In [ ]:
sensitivityAnalysis_REAL_MODEL2.getTotalOrderIndices()

As in this new sensitivity analysis are totally different from our first one, we have to check if either this analysis or the other is broken. For this we are first going to calculate the first and second order indices to see if something changes. 

In [ ]:
si_sec_2000_in = pd.read_csv('DOE_RESP/si_sec_doe2000_8291001288.csv', sep=';')
si_sec_2000_out_MD = pd.read_csv('DOE_RESP/si_sec_doe2000_8291001288_resp_MD.csv', sep=';',header=None)
si_sec_2000_out_VM = pd.read_csv('DOE_RESP/si_sec_doe2000_8291001288_resp_VM.csv', sep=';',header=None)
#Samples
sample_si_sec_2000_in = ot.Sample(si_sec_2000_in.values)
sample_si_sec_2000_in.setDescription(si_sec_2000_in.columns)

sample_si_sec_2000_out_MD = ot.Sample(si_sec_2000_out_MD.values)
sample_si_sec_2000_out_MD.setDescription(ot.Description.BuildDefault(len(si_sec_2000_out_MD.columns), 'MD_'))

sample_si_sec_2000_out_VM = ot.Sample(si_sec_2000_out_VM.values)
sample_si_sec_2000_out_VM.setDescription(ot.Description.BuildDefault(len(si_sec_2000_out_VM.columns), 'VM_'))

In [ ]:
N_sec_2000 = ereaseNanFromSample(sample_si_sec_2000_in , sample_si_sec_2000_out_MD, 2000, True)


In [ ]:
isValidSobolIndicesExperiment(sample_si_sec_2000_in, N_sec_2000,True)

In [ ]:
sensitivityAnalysis_REAL_MODEL2 = klfs.SobolKarhunenLoeveFieldSensitivityAlgorithm()
sensitivityAnalysis_REAL_MODEL2.setDesign(sample_si_sec_2000_in, sample_si_sec_2000_out_MD, N_sec_2000)
sensitivityAnalysis_REAL_MODEL2.setEstimator(ot.SaltelliSensitivityAlgorithm())
sensitivityAnalysis_REAL_MODEL2.setComputeSecondOrder(True)

In [ ]:
sensitivityAnalysis_REAL_MODEL2.inputDescription
sensitivityAnalysis_REAL_MODEL2.__nSobolIndices__
sensitivityAnalysis_REAL_MODEL2.size

In [ ]:
sensitivityAnalysis_REAL_MODEL2.getFirstOrderIndices()

In [ ]:
sensitivityAnalysis_REAL_MODEL2.getSecondOrderIndices()

In [ ]:
si_10000_in = pd.read_csv('./inp_10000.csv', sep=';')
si_10000_out_MD = pd.read_csv('./out_10000_MD.csv', sep=';')
si_10000_out_VM = pd.read_csv('./out_10000_VM.csv', sep=';')

In [ ]:
si_10000_in = pd.read_csv('./inp_10000.csv', sep=';')
si_10000_out_MD = pd.read_csv('./out_10000_MD.csv', sep=';',header=None)
si_10000_out_VM = pd.read_csv('./out_10000_VM.csv', sep=';',header=None)
#Samples
sample_si_10000_in = ot.Sample(si_10000_in.values)
sample_si_10000_in.setDescription(si_10000_in.columns)

sample_si_10000_out_MD = ot.Sample(si_10000_out_MD.values)
sample_si_10000_out_MD.setDescription(ot.Description.BuildDefault(len(si_10000_out_MD.columns), 'MD_'))

sample_si_10000_out_VM = ot.Sample(si_10000_out_VM.values)
sample_si_10000_out_VM.setDescription(ot.Description.BuildDefault(len(si_10000_out_VM.columns), 'VM_'))

In [ ]:
sensitivityAnalysis_10000 = klfs.SobolKarhunenLoeveFieldSensitivityAlgorithm()
sensitivityAnalysis_10000.setDesign(sample_si_10000_in, sample_si_10000_out_MD, 10000)
sensitivityAnalysis_10000.setEstimator(ot.SaltelliSensitivityAlgorithm())

In [ ]:
sensitivityAnalysis_10000.getFirstOrderIndices()

In [ ]:
sensitivityAnalysis_10000.getFirstOrderIndicesInterval()[0]

In [ ]:
sensitivityAnalysis_10000_2 = klfs.SobolKarhunenLoeveFieldSensitivityAlgorithm()
sensitivityAnalysis_10000_2.setDesign(sample_si_10000_in, sample_si_10000_out_MD, 10000)
sensitivityAnalysis_10000_2.setEstimator(ot.MartinezSensitivityAlgorithm())
sensitivityAnalysis_10000_2.getFirstOrderIndices()

In [ ]:
sensitivityAnalysis_10000_2.getFirstOrderIndicesInterval()[0]

In [ ]:
import spsa

In [ ]:
analysis2 = spsa.SobolIndicesStochasticProcessAlgorithm(np.array(sample_si_10000_out_MD),10000)
analysis2.getFirstOrderIndices()

In [ ]:
analysis2.getFirstOrderIndicesInterval()

In [ ]:
si_50000_in = pd.read_csv('./inp_50000.csv', sep=';')
si_50000_out_MD = pd.read_csv('./out_50000_MD.csv', sep=';',header=None)
si_50000_out_VM = pd.read_csv('./out_50000_VM.csv', sep=';',header=None)
#Samples
sample_si_50000_in = ot.Sample(si_50000_in.values)
sample_si_50000_in.setDescription(si_50000_in.columns)

sample_si_50000_out_MD = ot.Sample(si_50000_out_MD.values)
sample_si_50000_out_MD.setDescription(ot.Description.BuildDefault(len(si_50000_out_MD.columns), 'MD_'))

sample_si_50000_out_VM = ot.Sample(si_50000_out_VM.values)
sample_si_50000_out_VM.setDescription(ot.Description.BuildDefault(len(si_50000_out_VM.columns), 'VM_'))

In [ ]:
sensitivityAnalysis_50000 = klfs.SobolKarhunenLoeveFieldSensitivityAlgorithm()
sensitivityAnalysis_50000.setDesign(sample_si_50000_in, sample_si_50000_out_MD, 50000)
sensitivityAnalysis_50000.setEstimator(ot.SaltelliSensitivityAlgorithm())

In [ ]:
sensitivityAnalysis_50000.getFirstOrderIndices()

In [ ]:
sensitivityAnalysis_50000.getFirstOrderIndicesInterval()[0]

In [ ]:
sensitivityAnalysis_50000 = klfs.SobolKarhunenLoeveFieldSensitivityAlgorithm()
sensitivityAnalysis_50000.setDesign(sample_si_50000_in, sample_si_50000_out_MD, 50000)
sensitivityAnalysis_50000.setEstimator(ot.MartinezSensitivityAlgorithm())

In [ ]:
sensitivityAnalysis_50000.getFirstOrderIndices()

In [ ]:
sensitivityAnalysis_50000.getFirstOrderIndicesInterval()[0]

In [ ]:
analysis50000 = spsa.SobolIndicesStochasticProcessAlgorithm(np.array(sample_si_50000_out_MD),50000)
analysis50000.getFirstOrderIndices()

In [ ]:
analysis50000.getFirstOrderIndicesInterval()

### Sensitivity analysis of the metamodels 

Here we are going to do the same sensitivity analysis, but based on the metamodel of the bending beam model. 


In [ ]:
kriging_doe_200_MD.__kriging_metamodel__.getInputDimension()

In [ ]:
inp_50000 = pd.read_csv('./inp_50000.csv', sep=';')
sample_inp_50000 = ot.Sample(inp_50000.values)
sample_inp_50000.setDescription(inp_50000.columns)

In [ ]:
inp_50000

In [ ]:
resp_meatamodel50000 = kriging_doe_200_MD.__kriging_metamodel__(sample_inp_50000)

In [ ]:
sample_inp_50000

In [ ]:
sensitivityAnalysis_10000_2 = klfs.SobolKarhunenLoeveFieldSensitivityAlgorithm()
sensitivityAnalysis_10000_2.setDesign(sample_inp_50000, resp_meatamodel50000, 10000)
sensitivityAnalysis_10000_2.setEstimator(ot.MartinezSensitivityAlgorithm())
sensitivityAnalysis_10000_2.getFirstOrderIndices()

Refaire experience avec les mêmes samples pour le métamodele et le vrai modèle. 
(Plus vérification avec assert_equal() )
Nouveau Notebook plus concis + calculs R2 + graphes + introduire coef var dans params entrée + simplifier fonctions et mettre dans fichier à part. 
Cas tests :
  - D'abord kriegage (modifs taille DOE LHS) + écarts vrai modèle /metamodele
  - Sur paramètres entrée (variance, amplitude, scale... ) 
  - Sauvegarder chaque resultat
  - DF Pandas: ordonnées (indices de sobol) | abscisses : (Taille DOE due kriegage + taille plan d'experience) 
  - Tests chaos polynomial + visualisation + (analyser IS des sigma et des params) 